**Trabalho da Estacio - Tópicos de Big Data em Python**

Aluno: Núbia P. Serpa Andrade

Nota: Para funcionar corretamente, Faça o upload do arquivo csv chamado "Livros"
Coloque ele na pasta "content"

Nota: Caso não funcione o codigo abaixo, crie uma pasta chamada "templates" na pasta "content" e crie dentro dela um arquivo chamado "index.html", copie o codigo do arquivo de mesmo nome no meu repositorio do Github ou então, baixe do repositorio, faça o upload para dentro da pasta "templates"

In [ ]:
import os

os.makedirs("templates", exist_ok=True)


Programa

In [ ]:
!pip install pyspark pandas
!pip install pyngrok
from flask import Flask, render_template, jsonify
from pyngrok import ngrok
from pyspark.sql import SparkSession
import random

In [ ]:
spark = SparkSession.builder.appName("RecomendaçãoLivros").getOrCreate()
df = spark.read.csv("Livros.csv", header=True, inferSchema=True)
df.show()
port_no = 5000
app = Flask(__name__)
ngrok.set_auth_token("2lNafTj4EphmaNNYrOZgddrxruA_zuZo4XoGnwKLVFYmYX3Y")
public_url = ngrok.connect(port_no).public_url

In [ ]:
def recomendar_livros_aleatorio():
    generos = df.select("genero").distinct().rdd.map(lambda r: r[0]).collect()
    genero_aleatorio = random.choice(generos)

    livro_recomendado = df.filter(df.genero == genero_aleatorio).limit(1)
    resultado = livro_recomendado.select("genero", "nome_livro", "autor", "ano").collect()

    genero, nome_livro, autor, ano = resultado[0]
    return genero, nome_livro, autor, ano
@app.route("/")
def home():
    genero, nome_livro, autor, ano = recomendar_livros_aleatorio()
    return render_template("index.html", genero=genero, nome_livro=nome_livro, autor=autor, ano=ano)
@app.route("/nova_recomendacao")
def nova_recomendacao():
    genero, nome_livro, autor, ano = recomendar_livros_aleatorio()
    return jsonify(genero=genero, nome_livro=nome_livro, autor=autor, ano=ano)
print(f"{public_url}")
app.run(port=port_no)

Nota: Para visualizar, clique no primeiro link gerado e na pagina, aperte o botão roxo para confirmar